## Obtaining Data

In [1]:
import pandas as pd
import numpy as np
import re
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
# ML Libraries
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

In [2]:
data = pd.read_csv('data.csv')
data = data.astype(str)
tweets = pd.DataFrame(data['tweet_text'])

## Scrubbing/Cleaning Data

In [23]:
data.head(10) #preview of what data looks like

,tweet_text,emotion_in_tweet_is_directed_at,is_there_an_emotion_directed_at_a_brand_or_product
0,.@wesley83 I have a 3G iPhone. After 3 hrs twe...,iPhone,Negative emotion
1,@jessedee Know about @fludapp ? Awesome iPad/i...,iPad or iPhone App,Positive emotion
2,@swonderlin Can not wait for #iPad 2 also. The...,iPad,Positive emotion
3,@sxsw I hope this year's festival isn't as cra...,iPad or iPhone App,Negative emotion
4,@sxtxstate great stuff on Fri #SXSW: Marissa M...,Google,Positive emotion
5,@teachntech00 New iPad Apps For #SpeechTherapy...,nan,No emotion toward brand or product
6,nan,nan,No emotion toward brand or product
7,"#SXSW is just starting, #CTIA is around the co...",Android,Positive emotion
8,Beautifully smart and simple idea RT @madebyma...,iPad or iPhone App,Positive emotion
9,Counting down the days to #sxsw plus strong Ca...,Apple,Positive emotion


In [22]:
#addressing nan values
data = data[data['tweet_text'].notna()]
data = data.dropna()
data.dtypes

tweet_text                                            object
emotion_in_tweet_is_directed_at                       object
is_there_an_emotion_directed_at_a_brand_or_product    object
dtype: object

### Cleaning Text of Tweet

In [9]:
# Approach influenced by Machine Learning Course taken on Udemy
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []
for i in range(0, len(data['tweet_text'])):
    review = re.sub('[^a-zA-Z]', ' ', data['tweet_text'][i])
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    review = [ps.stem(word) for word in review if not word in set(stopwords.words('english'))]
    review = ' '.join(review)
    corpus.append(review)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jamaalsmith/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
data['tweet_text'] = corpus #so that dataframe has cleaned tweets

0       wesley g iphon hr tweet rise austin dead need ...
1       jessede know fludapp awesom ipad iphon app lik...
2                     swonderlin wait ipad also sale sxsw
3        sxsw hope year festiv crashi year iphon app sxsw
4       sxtxstate great stuff fri sxsw marissa mayer g...
                              ...                        
9283                             ipad everywher sxsw link
9284    wave buzz rt mention interrupt regularli sched...
9285    googl zeiger physician never report potenti ae...
9286    verizon iphon custom complain time fell back h...
9287          rt mention googl test check offer sxsw link
Name: tweet_text, Length: 9288, dtype: object

In [28]:
#Creation of Bag of Words
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 3000)
X = cv.fit_transform(corpus).toarray()
y = data.iloc[:, 1].values

In [ ]:
#making corpus
data['tweet_text'].to_list()
#corpus variable for all the test
corpus = data['tweet_text'].to_list()

In [ ]:
import nltk
nltk.download('punkt')

In [ ]:
##Tokenize data and then generate FreqDist
from nltk import word_tokenize
tokens = word_tokenize(','.join(map(str,corpus)))

In [ ]:
#FreqDist
from nltk import FreqDist
freq = FreqDist(tokens)
freq.most_common(100)

In [ ]:
#addressing stopwords
from nltk.corpus import stopwords
import string

print(stopwords.words('english'))
stopwords.remove('not')